# CGRA 2st Assignment

Diogo Seabra Mota Henriques de Gouveia - 2018279473

Manuel Alberto Dionísio dos Santos - 2019231352

In [ ]:
%%html
<!-- ============================================ Base Scripts ============================================ -->
<div id="Base Scripts">            
    <script id="deec_shader" src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
    <script id="deec_app" src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
    <script id="cgra_object" src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
    <script id="git" src='https://git.io/glm-js.min.js'></script>
</div>


<!-- ============================================ Base Scripts ============================================ -->
<div id="Shaders">
    <div id="DefaultShaders">
        <script id="my-vertex-shader" type="x-shader/x-vertex">
            precision mediump float;

            attribute  vec3 in_Position;
            attribute  vec3 in_Color;
            uniform mat4 MVP;

            varying  vec3 ex_Color;

            void main(void)
            {  
                gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

                ex_Color = in_Color;
            }
        </script>

        <script id="my-fragment-shader" type="x-shader/x-fragment">
            precision mediump float;

            varying  vec3 ex_Color;

            void main(void)
            {  
                gl_FragColor = vec4(ex_Color,1.0);   
            }
        </script>
    </div>
    
    <div id="TextureShaders">
        <script id="my-vertex-shaderT" type="x-shader/x-vertex">
            precision mediump float;

            attribute  vec3 in_Position;
            attribute  vec3 in_Color;
            attribute vec2 in_texcoords;
            uniform mat4 MVP;

            varying  vec3 ex_Color;
            varying  vec2 vTextureCoord;

            void main() {
            
                gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
                vTextureCoord = in_texcoords;
                ex_Color = in_Color;
            }
        </script>

        <script id="my-fragment-shaderT" type="x-shader/x-fragment">
            precision mediump float;
            varying  vec2 vTextureCoord;
            varying  vec3 ex_Color;

            uniform sampler2D uSampler;

            void main() {
                gl_FragColor = texture2D(uSampler, vTextureCoord);
            }
        </script>
    </div>

    <div id="TextureColorShaders">
        <script id="my-fragment-shaderTC" type="x-shader/x-fragment">
            precision mediump float;
            varying  vec2 vTextureCoord;
            varying  vec3 ex_Color;
            uniform vec3 un_Color;

            uniform sampler2D uSampler;

            void main() {
                gl_FragColor = vec4(un_Color * texture2D(uSampler, vTextureCoord).rgb,1.0);
            }
        </script>
    </div>
</div>


<!-- ============================================ CGRA Texture ============================================ -->
<div id="CGRA_Texture">
    <script>
        class CGRAtexture
        {
            constructor(glcontext)
            {
                this.gl = glcontext;
                this.textureid = this.gl.createTexture();
                this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
                this.level = 0;
                this.internalFormat = this.gl.RGBA;
                this.width = 2;
                this.height = 2;
                this.border = 0;
                this.srcFormat = this.gl.RGBA;
                this.srcType = this.gl.UNSIGNED_BYTE;
                this.pixels = new Uint8Array([255, 255, 255, 255, 
                                            0,0,0,255,  
                                            0,0,0,255, 
                                            255,255,255,255]);  
                this.gl.texImage2D(this.gl.TEXTURE_2D, this.level, this.internalFormat,
                        this.width, this.height, this.border, this.srcFormat, this.srcType,
                        this.pixels);
                this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_S, this.gl.CLAMP_TO_EDGE);
                this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_T, this.gl.CLAMP_TO_EDGE);
                //this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MAG_FILTER, this.gl.LINEAR);
                this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MAG_FILTER, this.gl.NEAREST);
                this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MIN_FILTER, this.gl.LINEAR);
            }
            
            loaded()
            {
                this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
                this.gl.texImage2D(this.gl.TEXTURE_2D, 0, this.internalFormat,
                        this.srcFormat, this.srcType, this.pixels);
                var ispowerof2 = ((this.pixels.width & 1) + (this.pixels.height & 1))==0;
                //console.log("width="+this.pixels.width+" height="+this.pixels.height);
                if (ispowerof2) {
                // Yes, it's a power of 2. Generate mipmaps.
                this.gl.generateMipmap(this.gl.TEXTURE_2D);
                } else {
                // No, it's not a power of 2. Turn off mipmaps and set
                // wrapping to clamp to edge
                this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_S, this.gl.CLAMP_TO_EDGE);
                this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_T, this.gl.CLAMP_TO_EDGE);
                    this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MAG_FILTER, this.gl.LINEAR);
                this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MIN_FILTER, this.gl.LINEAR);
                }
            }

            load(url)
            {
                this.pixels = new Image();
                this.pixels.src = url;
                this.pixels.onload = () => this.loaded();
            }
        }
    </script>
</div>


<!-- ============================================ Solids Definitions ============================================ -->
<canvas id="CubeCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="CubeApp">

class cube extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [1.0, 0.0, 0.0], col2 = col1)
    {
        // ===== Initialize the parent class
        super(glcontext); 
        
        // ===== Define COnstants
        this.numvertices = 36;
        this.size = 1;
        let dim = this.size/2;
        
        // ===== List vertices
        var verticesCube = [[ -dim, -dim,  dim],
                             [-dim,  dim,  dim],
                             [ dim,  dim,  dim],
                             [ dim, -dim,  dim],
                             [-dim, -dim, -dim],
                             [-dim,  dim, -dim],
                             [ dim,  dim, -dim],
                             [ dim, -dim, -dim]]; 

        // ===== Lookup table
        var vertices = [];
        var indexes = [
            0, 1, 2,  2, 3, 0,  // Front face
            6, 7, 3,  2, 3, 6,  // Right face
            6, 7, 4,  4, 5, 6,  // Back  face
            1, 0, 4,  4, 5, 1,  // Left  face
            1, 6, 2,  3, 4, 0,  // Up    face
            1, 6, 5,  3, 4, 7]; // Down  face
        
        // ===== Create structure to populate buffer
        for (var i = 0; i < indexes.length; i++)
        {
            vertices = vertices.concat(verticesCube[indexes[i]]);
        } 

        // ===== Assigning colors
        var colors = [];
        this.color1 = [col1, col1, col2].flat();
        this.color2 = [col2, col2, col1].flat();
        for (var i=0; i < (this.numvertices/3)/2; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        
        
        // ===== Creating and configuring buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Function to dinamically change cube colors
    setColors(newColor1, newColor2)
    {
        this.color1 = [newColor1, newColor1, newColor2].flat();
        this.color2 = [newColor2, newColor2, newColor1].flat();
        let colors = [];
        for (var i=0; i < (this.numvertices/3)/2; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
}

class cubeT extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        this.vColors = [];
        this.vPositions = [];
        this.texCoords = [];
        
        this.Index=0;
        this.positions = [
            glm.vec3( -0.5, -0.5,  0.5 ),
            glm.vec3( -0.5,  0.5,  0.5 ),
            glm.vec3(  0.5,  0.5,  0.5 ),
            glm.vec3(  0.5, -0.5,  0.5 ),
            glm.vec3( -0.5, -0.5, -0.5 ),
            glm.vec3( -0.5,  0.5, -0.5 ),
            glm.vec3(  0.5,  0.5, -0.5 ),
            glm.vec3(  0.5, -0.5, -0.5 )
        ];
        
        this.colors = [
            glm.vec3( 0.0, 0.0, 0.0 ),  // black
            glm.vec3( 1.0, 0.0, 0.0 ),  // red
            glm.vec3( 1.0, 1.0, 0.0 ),  // yellow
            glm.vec3( 0.0, 1.0, 0.0 ),  // green
            glm.vec3( 0.0, 0.0, 1.0 ),  // blue
            glm.vec3( 1.0, 0.0, 1.0 ),  // magenta
            glm.vec3( 1.0, 1.0, 1.0 ),  // white
            glm.vec3( 0.0, 1.0, 1.0 )   // cyan
        ];
        
        this.tcoords = [
            glm.vec2(0.0,0.0),
            glm.vec2(0.0,1.0),
            glm.vec2(1.0,1.0),
            glm.vec2(1.0,0.0)
        ];
        
      
        this.quad( 1, 0, 3, 2 );
        this.quad( 2, 3, 7, 6 );
        this.quad( 3, 0, 4, 7 );
        this.quad( 6, 5, 1, 2 );
        this.quad( 4, 5, 6, 7 );
        this.quad( 5, 4, 0, 1 );
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(this.vPositions);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        a = new Float32Array(this.vColors);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        a=new Float32Array(this.texCoords);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        //console.log("texc=" + this.texCoords+ "  a= "+ a + " len= "+a.length)
        
        this.numvertices = 36;
    }
    
     quad(a, b, c, d )
    {
  
        this.vColors = [].concat(this.vColors,this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[0].array);
        
        this.vColors = [].concat(this.vColors, this.colors[b].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[b].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[1].array);
                        
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[2].array);
      
        this.vColors = [].concat(this.vColors, this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[0].array);
       
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[2].array);
       
        this.vColors = [].concat(this.vColors,this.colors[d].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[d].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[3].array);

    }
    
     settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat,parentMat4 = glm.mat4(1.0)){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,parentMat4);
    }  
    
}

class cubeTC extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        this.color=glm.vec3(0,1,0);
        this.vColors = [];
        this.vPositions = [];
        this.texCoords = [];
        
        this.Index=0;
        this.positions = [
            glm.vec3( -0.5, -0.5,  0.5 ),
            glm.vec3( -0.5,  0.5,  0.5 ),
            glm.vec3(  0.5,  0.5,  0.5 ),
            glm.vec3(  0.5, -0.5,  0.5 ),
            glm.vec3( -0.5, -0.5, -0.5 ),
            glm.vec3( -0.5,  0.5, -0.5 ),
            glm.vec3(  0.5,  0.5, -0.5 ),
            glm.vec3(  0.5, -0.5, -0.5 )
        ];
        
        this.colors = [
            glm.vec3( 0.0, 0.0, 0.0 ),  // black
            glm.vec3( 1.0, 0.0, 0.0 ),  // red
            glm.vec3( 1.0, 1.0, 0.0 ),  // yellow
            glm.vec3( 0.0, 1.0, 0.0 ),  // green
            glm.vec3( 0.0, 0.0, 1.0 ),  // blue
            glm.vec3( 1.0, 0.0, 1.0 ),  // magenta
            glm.vec3( 1.0, 1.0, 1.0 ),  // white
            glm.vec3( 0.0, 1.0, 1.0 )   // cyan
        ];
        
        this.tcoords = [
            glm.vec2(0.0,0.0),
            glm.vec2(0.0,1.0),
            glm.vec2(1.0,1.0),
            glm.vec2(1.0,0.0)
        ];
        
      
        this.quad( 1, 0, 3, 2 );
        this.quad( 2, 3, 7, 6 );
        this.quad( 3, 0, 4, 7 );
        this.quad( 6, 5, 1, 2 );
        this.quad( 4, 5, 6, 7 );
        this.quad( 5, 4, 0, 1 );
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(this.vPositions);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        a = new Float32Array(this.vColors);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        a=new Float32Array(this.texCoords);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        //console.log("texc=" + this.texCoords+ "  a= "+ a + " len= "+a.length)
        
        this.numvertices = 36;
    }
    
     quad(a, b, c, d )
    {
  
        this.vColors = [].concat(this.vColors,this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[0].array);
        
        this.vColors = [].concat(this.vColors, this.colors[b].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[b].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[1].array);
                        
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[2].array);
      
        this.vColors = [].concat(this.vColors, this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[0].array);
       
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[2].array);
       
        this.vColors = [].concat(this.vColors,this.colors[d].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[d].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[3].array);

    }
    
     settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    setcolor(color){
            this.color = glm.vec3(color);
    }

    drawit(viewMat, projectionMat,parentMat4 = glm.mat4(1.0)){
        this.shaderprog.startUsing();
        var colorLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,
                                                          "un_Color");    
        this.gl.uniform3fv(colorLocation, this.color.array);
       
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
       
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,parentMat4);
    }  
    
}

class CubeApp extends DEECapp
{
    counter = 0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Instantiate a cube
        this.myCube = new cube(this.gl, [1, 0, 0]);
        this.myCube.setShader(this.shaderprog);
        
         var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
   
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("CubeTex.png");
        
        this.myCubeT = new cubeT(this.gl);
        this.myCubeT.settexture(this.texture);
        this.myCubeT.setShader(this.shaderprogT);
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,100);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(3, 3, 5): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(0,0,5),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;

        this.shaderprog.startUsing();
                
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Translate
        var translation = glm.vec3(-1,Math.sin(this.counter/5),-3/5);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*5);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.myCube.setModelTransformation(model);      
        
        // ===== Draw Cube
        this.myCube.drawit(this.viewM,this.projectionM);
        
         // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Translate
        var translation = glm.vec3(1,Math.sin(this.counter/5),-3/5);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*5);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.myCubeT.setModelTransformation(model);      
        
        // ===== Draw Cube
        this.myCubeT.drawit(this.viewM,this.projectionM);
        
        // ===== Play with colors
        var speed = this.counter/10;
        
    }
}

var app = new CubeApp('CubeCanvas');

app.run();
</script>
# Sphere

To create a sphere of diameter 1, we created N slices of it, composed of N quadrilateral's each (2 triangles for each quadrilateral, except on top and bottom ends, that have only 1 each).

We pushd each slice into the vertex buffer and at the same time the corresponding colors to the color buffer

Finally, we instantiated a sphere object and moved the camera around it.


<canvas id="SphereCanvas" width="800" height="800" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="SphereApp">

class sphere extends CGRAobject
{
    center = glm.vec4(0, 0, 0, 1); 
    north_pole = glm.vec4(0, 0.5, 0, 1); 
    
    // ===== Constructor
    constructor(glcontext, col = [1.0, 1.0, 1.0], col2 = col, faces = 10)
    {
        // Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        // ===== Number of vertices
        this.numvertices = (6*this.sides + (6*this.sides)*(this.sides-2));
        this.length = 1; this.radius = 0.5;
        
        // ===== Create circunferences (360/N sides)
        let slices = [];   
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides, aux = [];
            for (let j = 0; j < this.sides-1; j++)
            {     
                aux.push(
                    [(Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.cos(theta),
                     Math.cos((j+1)*Math.PI/this.sides)*this.radius,
                    (Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.sin(theta)]);
            }   
            slices.push(aux);
        }
        slices.push([[0,this.radius,0],[0,-1*this.radius,0]]);
        
        
        var vertices = [];
        var colors = [];
        
        // ===== Create vertex and color data to populate buffers
        for (let i = 0; i < this.sides; i++)
        {            
            // Top side
            vertices.push(slices[this.sides][0]);
            vertices.push(slices[i][0]);
            vertices.push(slices[(i+1)%this.sides][0]);
    
            
            colors.push([col,col,col].flat());
            
            // Middle sections
            for (let j = 0; j < this.sides - 2; j++)
            {
                vertices.push(slices[i][j]);
                vertices.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[i][j+1]);
                
               
                colors.push([col,col,col].flat());

                vertices.push(slices[i][j+1]);
                vertices.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[(i+1)%this.sides][j+1]);

                
                colors.push([col,col,col].flat());
            }
            
            // Down side
            vertices.push(slices[i][this.sides - 2]);
            vertices.push(slices[(i+1)%this.sides][this.sides - 2]);
            vertices.push(slices[this.sides][1]);
            colors.push([col,col,col].flat());
            
            
        }
        
        // ===== Populate Buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        
    }
    
    // ===== Funcion to register center and north of sphere (WIP)
    update_pos(parentMat4 = glm.mat4(1.0))
    {
        var globalMat = parentMat4['*'](this.modelMat);
        
        //console.log(globalMat);
        
        this.center = globalMat['*'](this.center);
        //console.log("Center: " + this.center);
        this.north_pole = globalMat['*'](this.north_pole);
        //console.log("North: " + this.center);
    }
}

class sphereT extends CGRAobject
{
    center = glm.vec4(0, 0, 0, 1); 
    north_pole = glm.vec4(0, 0.5, 0, 1); 
    
    // ===== Constructor
    constructor(glcontext, col = [1.0, 1.0, 1.0], faces = 10)
    {
        // Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        // ===== Number of vertices
        this.numvertices = (6*this.sides + (6*this.sides)*(this.sides-2));
        this.length = 1; this.radius = 0.5;
        
        // ===== Create circunferences (360/N sides)
        let slices = [];  
        
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides, aux = [];
            for (let j = 0; j < this.sides-1; j++)
            {     
                aux.push(
                    [(Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.cos(theta),
                     Math.cos((j+1)*Math.PI/this.sides)*this.radius,
                    (Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.sin(theta)]);
            }
            slices.push(aux);
        }
        slices.push([[0,this.radius,0],[0,-1*this.radius,0]]);
        

        var vertices = [];
        var texCoords = [];
        var colors = [];
        
        // ===== Create vertex and color data to populate buffers
        for (let i = 0; i < this.sides; i++)
        {            
            // Top side
            vertices.push(slices[this.sides][0]); // topo
            vertices.push(slices[i][0]); // esquerda
            vertices.push(slices[(i+1)%this.sides][0]); // direita
            
            texCoords.push([(2*i+1)/(this.sides*2),1]); // topo
            texCoords.push([(2*i)/(this.sides*2), 1-1/this.sides]); // esquerda
            texCoords.push([(2*i+2)/(this.sides*2),1-1/this.sides]);  // direita
            

            colors.push([col,col,col].flat());
            
            // Middle sections
            for (let j = 0; j < this.sides - 2; j++)
            {
                vertices.push(slices[i][j]);
                vertices.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[i][j+1]);
                
                texCoords.push([(2*i)/(this.sides*2), 1-(j+1)/this.sides]); //a
                texCoords.push([((2*i)+2)/(this.sides*2), 1-(j+1)/this.sides]); //b
                texCoords.push([(2*i)/(this.sides*2), 1-(j+2)/this.sides]); //c
                

                colors.push([col,col,col].flat());

                vertices.push(slices[i][j+1]);
                vertices.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[(i+1)%this.sides][j+1]);
                
                texCoords.push([(2*i)/(this.sides*2), 1-(j+2)/this.sides]); //c
                texCoords.push([((2*i)+2)/(this.sides*2), 1-(j+1)/this.sides]); //b
                texCoords.push([((2*i)+2)/(this.sides*2), 1-(j+2)/this.sides]); //d
                

                
                colors.push([col,col,col].flat());
            }
            
            // Down side
            vertices.push(slices[i][this.sides - 2]); //left
            vertices.push(slices[(i+1)%this.sides][this.sides - 2]); //right
            vertices.push(slices[this.sides][1]);  //bottom
            
            
            texCoords.push([(2*i)/(this.sides*2), 1/this.sides]); // esquerda
            texCoords.push([(2*i+2)/(this.sides*2),1/this.sides]);  // direita
            texCoords.push([(2*i+1)/(this.sides*2),0]); // bottom

       
            colors.push([col,col,col].flat());
            
            
        }
        
        
        
                
              // ===== Populate vertex and color buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texCoords.flat()), this.gl.STATIC_DRAW);
    
        
    }
    
        settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat,parentMat4 = glm.mat4(1.0)){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                                "in_texcoords");
            
              this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordbuffer);
              this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                                2, // number of elements per attribute
                                this.gl.FLOAT,  // Type of elements
                                false,  // 
                                0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
              this.gl.enableVertexAttribArray(this.texcoordsLocation);
              this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
              this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
              this.gl.uniform1i(this.textureLocation, 0);
              
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,parentMat4);
    }  
    
    // ===== Funcion to register center and north of sphere (WIP)
    update_pos(parentMat4 = glm.mat4(1.0))
    {
        var globalMat = parentMat4['*'](this.modelMat);
        
        //console.log(globalMat);
        
        this.center = globalMat['*'](this.center);
        //console.log("Center: " + this.center);
        this.north_pole = globalMat['*'](this.north_pole);
        //console.log("North: " + this.center);
    }

    

}



class sphereTC extends CGRAobject
{
    center = glm.vec4(0, 0, 0, 1); 
    north_pole = glm.vec4(0, 0.5, 0, 1); 
    
    // ===== Constructor
    constructor(glcontext, col = [1.0, 1.0, 1.0], faces = 10)
    {
        // Initialize the parent class
        super(glcontext); 
        this.color=glm.vec3(0,1,0);
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        // ===== Number of vertices
        this.numvertices = (6*this.sides + (6*this.sides)*(this.sides-2));
        this.length = 1; this.radius = 0.5;
        
        // ===== Create circunferences (360/N sides)
        let slices = [];  
        
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides, aux = [];
            for (let j = 0; j < this.sides-1; j++)
            {     
                aux.push(
                    [(Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.cos(theta),
                     Math.cos((j+1)*Math.PI/this.sides)*this.radius,
                    (Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.sin(theta)]);
            }
            slices.push(aux);
        }
        slices.push([[0,this.radius,0],[0,-1*this.radius,0]]);
        

        var vertices = [];
        var texCoords = [];
        var colors = [];
        
        // ===== Create vertex and color data to populate buffers
        for (let i = 0; i < this.sides; i++)
        {            
            // Top side
            vertices.push(slices[this.sides][0]); // topo
            vertices.push(slices[i][0]); // esquerda
            vertices.push(slices[(i+1)%this.sides][0]); // direita
            
            texCoords.push([(2*i+1)/(this.sides*2),1]); // topo
            texCoords.push([(2*i)/(this.sides*2), 1-1/this.sides]); // esquerda
            texCoords.push([(2*i+2)/(this.sides*2),1-1/this.sides]);  // direita
            

            colors.push([col,col,col].flat());
            
            // Middle sections
            for (let j = 0; j < this.sides - 2; j++)
            {
                vertices.push(slices[i][j]);
                vertices.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[i][j+1]);
                
                texCoords.push([(2*i)/(this.sides*2), 1-(j+1)/this.sides]); //a
                texCoords.push([((2*i)+2)/(this.sides*2), 1-(j+1)/this.sides]); //b
                texCoords.push([(2*i)/(this.sides*2), 1-(j+2)/this.sides]); //c
                

                colors.push([col,col,col].flat());

                vertices.push(slices[i][j+1]);
                vertices.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[(i+1)%this.sides][j+1]);
                
                texCoords.push([(2*i)/(this.sides*2), 1-(j+2)/this.sides]); //c
                texCoords.push([((2*i)+2)/(this.sides*2), 1-(j+1)/this.sides]); //b
                texCoords.push([((2*i)+2)/(this.sides*2), 1-(j+2)/this.sides]); //d
                

                
                colors.push([col,col,col].flat());
            }
            
            // Down side
            vertices.push(slices[i][this.sides - 2]); //left
            vertices.push(slices[(i+1)%this.sides][this.sides - 2]); //right
            vertices.push(slices[this.sides][1]);  //bottom
            
            
            texCoords.push([(2*i)/(this.sides*2), 1/this.sides]); // esquerda
            texCoords.push([(2*i+2)/(this.sides*2),1/this.sides]);  // direita
            texCoords.push([(2*i+1)/(this.sides*2),0]); // bottom

       
            colors.push([col,col,col].flat());
            
            
        }
        
        
        
                
              // ===== Populate vertex and color buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texCoords.flat()), this.gl.STATIC_DRAW);
    
        
    }
    
        settexture(cgratex){
            this.textureid = cgratex.textureid;
    }

    setcolor(color){
            this.color = glm.vec3(color);
    }
    
    drawit(viewMat, projectionMat,parentMat4 = glm.mat4(1.0)){
        this.shaderprog.startUsing();
        var colorLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,
                                                          "un_Color");    
        this.gl.uniform3fv(colorLocation, this.color.array);
       
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
       
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
              
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,parentMat4);
    }  
    
    // ===== Funcion to register center and north of sphere (WIP)
    update_pos(parentMat4 = glm.mat4(1.0))
    {
        var globalMat = parentMat4['*'](this.modelMat);
        
        //console.log(globalMat);
        
        this.center = globalMat['*'](this.center);
        //console.log("Center: " + this.center);
        this.north_pole = globalMat['*'](this.north_pole);
        //console.log("North: " + this.center);
    }

    

}


class SphereApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9,0.9,0.9,1.0);
        
        // ===== Create sphere
        this.my_Sphere = new sphere(this.gl,[0.95, 0.95, 0], [0,0,0] , 20);
        this.my_Sphere.setShader(this.shaderprog);
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
   
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("HeadTex.png");
        
        this.my_SphereT = new sphereT(this.gl,[0.95, 0.95, 0] , 20);
        this.my_SphereT.settexture(this.texture);
        this.my_SphereT.setShader(this.shaderprogT);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(5,0,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
                // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
        var translation = glm.vec3(0,0,-1);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 0.0, 1.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_Sphere.setModelTransformation(model); 
        
        // ==== Draw
        this.my_Sphere.drawit(this.viewM,this.projectionM);

        
        var model = glm.mat4(glm.mat3(1.0));
                var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
         var translation = glm.vec3(0,0,1);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 0.0, 1.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_SphereT.setModelTransformation(model);
        this.my_SphereT.drawit(this.viewM,this.projectionM);
        
        
        
      
    }
}

var app = new SphereApp('SphereCanvas');

app.run();
</script>
# Cylinder / Prism

To create a cylinder of base 1, we created 2 bases (up and down) and conected each "face" of them with 2 triangles. In terms of division of the bases, the procedure is similar to the sphere. In this way, is possible to create solids from 3 to infinite faces. That means we can create a prism (3 faces), a "paralelepípedo"/cube (4 faces) or a clinder (5 to inf faces).

We also created a method to dinamically change the cylinder colors.

Finally, we instantiated a cylinder object and rotated it around the Z axis.


<canvas id="CylinderCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="CylinderApp">


class cylinder extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 1.0], faces = 10)
    {
        // Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*6 + this.sides*3*2;
        this.length = 1; this.radius = 0.5;
        
        // ===== Calculate top and bottom bases coordinates
        let up_side = [], down_side = [];   
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            up_side.push([this.radius*Math.cos(theta),
                          (this.length)/2,
                          this.radius*Math.sin(theta)]);
            
            down_side.push([this.radius*Math.cos(theta),
                            -1*(this.length)/2,
                            this.radius*Math.sin(theta)]);
        }
        
        // ===== Calculate vertices
        var vertices = [];
        for (let i = 0; i < this.sides; i++)
        {   
            vertices.push(up_side[i]);
            vertices.push(up_side[(i+1)%this.sides]);
            vertices.push(down_side[i]);
            
            vertices.push(down_side[i]);
            vertices.push(down_side[(i+1)%this.sides]);
            vertices.push(up_side[(i+1)%this.sides]);
            
            vertices.push(up_side[i]);
            vertices.push(up_side[(i+1)%this.sides]);
            vertices.push([0,(this.length)/2,0]);
            
            vertices.push(down_side[i]);
            vertices.push(down_side[(i+1)%this.sides]);
            vertices.push([0,-1*(this.length)/2,0]);
        }

        // ===== Assign colors
        var colors = [];
        this.color1 = [col1, col1, col2].flat();
        this.color2 = [col2, col2, col1].flat();
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        /*var alpha = 0;
        for (let i=0; i < this.numvertices; i++)
        {
            colors.push(col1[0]*alpha + (1*alpha)*col2[0]);
            colors.push(col1[1]*alpha + (1*alpha)*col2[1]);
            colors.push(col1[2]*alpha + (1*alpha)*col2[2]);
            alpha += 1.0/this.numvertices;
        }*/
        
        // ===== Populate vertex and color buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Function to dynamically change the cylinder colors
    setColors(newColor1, newColor2)
    {
        this.color1 = [newColor1, newColor1, newColor2].flat();
        this.color2 = [newColor2, newColor2, newColor1].flat();
        let colors = [];
        for (var i=0; i < (this.numvertices/3)/2; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
}


class cylinderT extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 1.0], faces = 10)
    {
        // Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*6 + this.sides*3*2;
        this.length = 1; this.radius = 0.5;
        
        // ===== Calculate top and bottom bases coordinates
        let up_side = [], down_side = [], circleT =[];
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            up_side.push([this.radius*Math.cos(theta),
                          (this.length)/2,
                          this.radius*Math.sin(theta)]);
            
            down_side.push([this.radius*Math.cos(theta),
                            -1*(this.length)/2,
                            this.radius*Math.sin(theta)]);
            
            circleT.push([0.25*Math.cos(theta)+0.25,
                            0.25*Math.sin(theta)+0.25]);
            
            
        }
        
        // ===== Calculate vertices
        var vertices = [];
        var texCoords = [];
        for (let i = 0; i < this.sides; i++)
        {   
            
            //LAT
            vertices.push(up_side[i]);
            vertices.push(up_side[(i+1)%this.sides]);
            vertices.push(down_side[i]);
 
            texCoords.push([0.5+(i)*0.5/this.sides, 0.5]);
            texCoords.push([0.5+(i+1)*0.5/this.sides, 0.5]);
            texCoords.push([0.5+(i)*0.5/this.sides, 0]);           
            
            vertices.push(down_side[i]);
            vertices.push(down_side[(i+1)%this.sides]);
            vertices.push(up_side[(i+1)%this.sides]);
            
            texCoords.push([0.5+(i)*0.5/this.sides, 0]);
            texCoords.push([0.5+(i+1)*0.5/this.sides, 0]);
            texCoords.push([0.5+(i+1)*0.5/this.sides, 0.5]);
            
            //TOP BASE
            
            vertices.push(up_side[i]);
            vertices.push(up_side[(i+1)%this.sides]);
            vertices.push([0,(this.length)/2,0]);
            
            texCoords.push(circleT[i]);
            texCoords.push(circleT[(i+1)%this.sides]);
            texCoords.push([0.25,0.25]);
            
            // DOWN BASE
            
            vertices.push(down_side[i]);
            vertices.push(down_side[(i+1)%this.sides]);
            vertices.push([0,-1*(this.length)/2,0]);
            
            texCoords.push(circleT[i]);
            texCoords.push(circleT[(i+1)%this.sides]);
            texCoords.push([0.25,0.25]);
            

        }

        // ===== Assign colors
        var colors = [];
        this.color1 = [col1, col1, col2].flat();
        this.color2 = [col2, col2, col1].flat();
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }

        
        // ===== Populate vertex and color buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texCoords.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Function to dynamically change the cylinder colors
    setColors(newColor1, newColor2)
    {
        this.color1 = [newColor1, newColor1, newColor2].flat();
        this.color2 = [newColor2, newColor2, newColor1].flat();
        let colors = [];
        for (var i=0; i < (this.numvertices/3)/2; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
            
    drawit(viewMat, projectionMat, parentMat4 = glm.mat4(1.0)){
              this.shaderprog.startUsing();
              this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                                "in_texcoords");
            
              this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordbuffer);
              this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                                2, // number of elements per attribute
                                this.gl.FLOAT,  // Type of elements
                                false,  // 
                                0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
              this.gl.enableVertexAttribArray(this.texcoordsLocation);
              this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
              this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
              this.gl.uniform1i(this.textureLocation, 0);
              
        // the parent method does the rest
        super.drawit(viewMat,projectionMat, parentMat4);
            } 
}


class cylinderTC extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 1.0], faces = 10)
    {
        // Initialize the parent class
        super(glcontext); 

        this.color=glm.vec3(0,1,0);
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*6 + this.sides*3*2;
        this.length = 1; this.radius = 0.5;
        
        // ===== Calculate top and bottom bases coordinates
        let up_side = [], down_side = [], circleT =[];
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            up_side.push([this.radius*Math.cos(theta),
                          (this.length)/2,
                          this.radius*Math.sin(theta)]);
            
            down_side.push([this.radius*Math.cos(theta),
                            -1*(this.length)/2,
                            this.radius*Math.sin(theta)]);
            
            circleT.push([0.25*Math.cos(theta)+0.25,
                            0.25*Math.sin(theta)+0.25]);
            
            
        }
        
        // ===== Calculate vertices
        var vertices = [];
        var texCoords = [];
        for (let i = 0; i < this.sides; i++)
        {   
            
            //LAT
            vertices.push(up_side[i]);
            vertices.push(up_side[(i+1)%this.sides]);
            vertices.push(down_side[i]);
 
            texCoords.push([0.5+(i)*0.5/this.sides, 0.5]);
            texCoords.push([0.5+(i+1)*0.5/this.sides, 0.5]);
            texCoords.push([0.5+(i)*0.5/this.sides, 0]);           
            
            vertices.push(down_side[i]);
            vertices.push(down_side[(i+1)%this.sides]);
            vertices.push(up_side[(i+1)%this.sides]);
            
            texCoords.push([0.5+(i)*0.5/this.sides, 0]);
            texCoords.push([0.5+(i+1)*0.5/this.sides, 0]);
            texCoords.push([0.5+(i+1)*0.5/this.sides, 0.5]);
            
            //TOP BASE
            
            vertices.push(up_side[i]);
            vertices.push(up_side[(i+1)%this.sides]);
            vertices.push([0,(this.length)/2,0]);
            
            texCoords.push(circleT[i]);
            texCoords.push(circleT[(i+1)%this.sides]);
            texCoords.push([0.25,0.25]);
            
            // DOWN BASE
            
            vertices.push(down_side[i]);
            vertices.push(down_side[(i+1)%this.sides]);
            vertices.push([0,-1*(this.length)/2,0]);
            
            texCoords.push(circleT[i]);
            texCoords.push(circleT[(i+1)%this.sides]);
            texCoords.push([0.25,0.25]);
            

        }

        // ===== Assign colors
        var colors = [];
        this.color1 = [col1, col1, col2].flat();
        this.color2 = [col2, col2, col1].flat();
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }

        
        // ===== Populate vertex and color buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texCoords.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Function to dynamically change the cylinder colors
    setColors(newColor1, newColor2)
    {
        this.color1 = [newColor1, newColor1, newColor2].flat();
        this.color2 = [newColor2, newColor2, newColor1].flat();
        let colors = [];
        for (var i=0; i < (this.numvertices/3)/2; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }

    setcolor(color){
            this.color = glm.vec3(color);
    }
            
    drawit(viewMat, projectionMat, parentMat4 = glm.mat4(1.0)){
              this.shaderprog.startUsing();
              var colorLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,
                                                          "un_Color");    
        this.gl.uniform3fv(colorLocation, this.color.array);
       
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
       
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        
              
        // the parent method does the rest
        super.drawit(viewMat,projectionMat, parentMat4);
            } 
}


class CylinderApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Create cylinder
        this.my_Cylinder = new cylinder(this.gl);
        this.my_Cylinder.setShader(this.shaderprog);
        
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
   
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("CyTex.png");
        
        this.my_CylinderT = new cylinderT(this.gl,[0.9, 0, 0] ,[1, 1, 0],10);
        this.my_CylinderT.settexture(this.texture);
        this.my_CylinderT.setShader(this.shaderprogT);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(5,0,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        this.shaderprog.startUsing();
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
        var translation = glm.vec3(0,0,-1);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 0.0, 1.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_Cylinder.setModelTransformation(model); 
        
        // ==== Draw
        this.my_Cylinder.drawit(this.viewM,this.projectionM);

        
        var model = glm.mat4(glm.mat3(1.0));
                var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
         var translation = glm.vec3(0,0,1);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 0.0, 1.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_CylinderT.setModelTransformation(model);
        
        // ===== Draw cone
        this.my_CylinderT.drawit(this.viewM,this.projectionM);
        
    }
}

var app = new CylinderApp('CylinderCanvas');

app.run();
</script>
# Cone

The procedure to create the cone is in everything similar to the one to create a cylinder, but without the top base.


<canvas id="ConeCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="ConeApp">

class cone extends CGRAobject
{
    // ===== Contructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 0.0], faces = 10)
    {
        // ===== Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*3*2;
        this.length = 1; this.radius = 0.5;
        
        // ===== Calculate base
        let base = [];   
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            base.push(
                [this.radius*Math.cos(theta),
                 -1*(this.length)/2,
                this.radius*Math.sin(theta)]);
        }
        
        // ===== Calculate vertices
        var vertices = [];
        for (let i = 0; i < this.sides; i++)
        {   
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0,-1*(this.length)/2,0]);
            
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0,(this.length)/2,0]);
        }
        
        // ===== Assign colors
        this.Cone_colors = [col1, col1, col2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Cone_colors);
        }
        

        
        // ===== Populate buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        

    }
    
    // Function to dynamically change colors
    setColors(newColor1, newColor2)
    {
        this.Cone_colors = [newColor1, newColor1, newColor2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Cone_colors);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
}



class coneT extends CGRAobject
{
    // ===== Contructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 0.0], faces = 10)
    {
        // ===== Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*3*2;
        this.length = 1; this.radius = 0.5;
        
        // ===== Calculate base
        let base = [];
        var texCoords = [];
        let baseT = [];
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            base.push(
                [this.radius*Math.cos(theta),
                 -1*(this.length)/2,
                this.radius*Math.sin(theta)]);
            baseT.push(
                [this.radius*Math.cos(theta)+0.5,
                this.radius*Math.sin(theta)+0.5]);
        }
        
        // ===== Calculate vertices
        var vertices = [];
        
        
        for (let i = 0; i < this.sides; i++)
        {   
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0,-1*(this.length)/2,0]);
            
            texCoords.push(baseT[i]);
            texCoords.push(baseT[(i+1)%this.sides]);
            texCoords.push([0.5,0.5]);
            
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0,(this.length)/2,0]);
            
            texCoords.push(baseT[i]);
            texCoords.push(baseT[(i+1)%this.sides]);
            texCoords.push([0.5,0.5]);
        }
        
        // ===== Assign colors
        this.Cone_colors = [col1, col1, col2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Cone_colors);
        }
        
        
        // ===== Populate buffers

        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        
                
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texCoords.flat()), this.gl.STATIC_DRAW);
            
        

    }
    
    // Function to dynamically change colors
    setColors(newColor1, newColor2)
    {
        this.Cone_colors = [newColor1, newColor1, newColor2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Cone_colors);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    
            
            settexture(cgratex){
                    this.textureid = cgratex.textureid;
            }
            
            drawit(viewMat, projectionMat, parentMat4 = glm.mat4(1.0)){
              this.shaderprog.startUsing();
              this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                                "in_texcoords");
            
              this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordbuffer);
              this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                                2, // number of elements per attribute
                                this.gl.FLOAT,  // Type of elements
                                false,  // 
                                0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
              this.gl.enableVertexAttribArray(this.texcoordsLocation);
              this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
              this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
              this.gl.uniform1i(this.textureLocation, 0);
              
              // the parent method does the rest
              super.drawit(viewMat,projectionMat, parentMat4);
            } 
}

class coneTC extends CGRAobject
{
    // ===== Contructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 0.0], faces = 10)
    {
        // ===== Initialize the parent class
        super(glcontext); 
        this.color=glm.vec3(0,1,0);
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*3*2;
        this.length = 1; this.radius = 0.5;
        
        // ===== Calculate base
        let base = [];
        var texCoords = [];
        let baseT = [];
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            base.push(
                [this.radius*Math.cos(theta),
                 -1*(this.length)/2,
                this.radius*Math.sin(theta)]);
            baseT.push(
                [this.radius*Math.cos(theta)+0.5,
                this.radius*Math.sin(theta)+0.5]);
        }
        
        // ===== Calculate vertices
        var vertices = [];
        
        
        for (let i = 0; i < this.sides; i++)
        {   
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0,-1*(this.length)/2,0]);
            
            texCoords.push(baseT[i]);
            texCoords.push(baseT[(i+1)%this.sides]);
            texCoords.push([0.5,0.5]);
            
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0,(this.length)/2,0]);
            
            texCoords.push(baseT[i]);
            texCoords.push(baseT[(i+1)%this.sides]);
            texCoords.push([0.5,0.5]);
        }
        
        // ===== Assign colors
        this.Cone_colors = [col1, col1, col2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Cone_colors);
        }
        
        
        // ===== Populate buffers

        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        
                
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texCoords.flat()), this.gl.STATIC_DRAW);
            
        

    }
    
    // Function to dynamically change colors
    setColors(newColor1, newColor2)
    {
        this.Cone_colors = [newColor1, newColor1, newColor2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Cone_colors);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    setcolor(color){
            this.color = glm.vec3(color);
    }
            
            settexture(cgratex){
                    this.textureid = cgratex.textureid;
            }
            
            drawit(viewMat, projectionMat, parentMat4 = glm.mat4(1.0)){
              this.shaderprog.startUsing();
              var colorLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,
                                                          "un_Color");    
        this.gl.uniform3fv(colorLocation, this.color.array);
       
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
       
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
              
              // the parent method does the rest
              super.drawit(viewMat,projectionMat, parentMat4);
            } 
}






class ConeApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Create cone
        this.my_cone = new cone(this.gl, [0.9, 0, 0] ,[1, 1, 0], 10);
        this.my_cone.setShader(this.shaderprog);
        

        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
   
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("ConeTex.png");
        
        this.my_coneT = new coneT(this.gl,[0.9, 0, 0] ,[1, 1, 0],30);
        this.my_coneT.settexture(this.texture);
        this.my_coneT.setShader(this.shaderprogT);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(5,0,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        
        ////////////////// Transform/Animate Couple //////////////////
        var model = glm.mat4(glm.mat3(1.0));
         var translation = glm.vec3(0,0,-1);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 0.0, 1.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_cone.setModelTransformation(model);
        
        // ===== Draw cone
        this.my_cone.drawit(this.viewM,this.projectionM);
        
        var model = glm.mat4(glm.mat3(1.0));
         var translation = glm.vec3(0,0,1);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 0.0, 1.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_coneT.setModelTransformation(model);
        
        // ===== Draw cone
       this.my_coneT.drawit(this.viewM,this.projectionM);
    }
}

var app = new ConeApp('ConeCanvas');

app.run();
</script>
# Disc

The procedure to create the cone is in everything similar to the one to create a cylinder or a cone, but just needs to create a base.


<canvas id="DiscCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="DiscApp">

class disc extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 0.0], faces = 10)
    {
        // ===== Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*3;
        this.radius = 0.5;
        
        // ===== Calculate vertices
        let base = [];   
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            base.push(
                [this.radius*Math.cos(theta),
                 0.0,
                this.radius*Math.sin(theta)]);
        }
        
        // ===== Assign vertices
        var vertices = [];
        for (let i = 0; i < this.sides; i++)
        {   
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0,0,0]);
        }
        
        // ===== Assign colors
        this.Disc_colors = [col1, col1, col2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Disc_colors);
        }        
        
        // ===== Populate buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Function to dinamically change colors of disc
    setColors(newColor1, newColor2)
    {
        this.Disc_colors = [newColor1, newColor1, newColor2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Disc_colors);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
}


class discT extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [0.0, 1.0, 0.0], col2 = [0.0, 0.0, 0.0], faces = 10)
    {
        // ===== Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        this.numvertices = this.sides*3;
        this.radius = 0.5;
        
        // ===== Calculate vertices
        let base = []; 
        let baseT = [];
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides;
            base.push(
                [this.radius*Math.cos(theta),
                 0.0,
                this.radius*Math.sin(theta)]);
            baseT.push(
                [this.radius*Math.cos(theta)+0.5,
                this.radius*Math.sin(theta)+0.5]);
        }
        
        // ===== Assign vertices
        var vertices = [], texCoords = [];
        for (let i = 0; i < this.sides; i++)
        {   
            vertices.push(base[i]);
            vertices.push(base[(i+1)%this.sides]);
            vertices.push([0.0,0.0,0.0]);
            
            texCoords.push(baseT[i]);
            texCoords.push(baseT[(i+1)%this.sides]);
            texCoords.push([0.5,0.5]);
        }
        
        // ===== Assign colors
        this.Disc_colors = [col1, col1, col2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Disc_colors);
        }        
        
        // ===== Populate buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        
        this.texcoordbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texCoords.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Function to dinamically change colors of disc
    setColors(newColor1, newColor2)
    {
        this.Disc_colors = [newColor1, newColor1, newColor2].flat();
        let colors = [];
        for (let i=0; i < (this.sides*6)/3; i++)
        {
            colors.push(this.Disc_colors);
        }
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    settexture(cgratex){
                    this.textureid = cgratex.textureid;
            }
            
            drawit(viewMat, projectionMat){
              this.shaderprog.startUsing();
              this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                                "in_texcoords");
            
              this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordbuffer);
              this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                                2, // number of elements per attribute
                                this.gl.FLOAT,  // Type of elements
                                false,  // 
                                0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                                0); // Offset from the begining of a single vertex to this attribute
              this.gl.enableVertexAttribArray(this.texcoordsLocation);
              this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
              this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
              this.gl.uniform1i(this.textureLocation, 0);
              
              // the parent method does the rest
              super.drawit(viewMat,projectionMat);
            } 
}


class DiscApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform other initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9,0.9,0.9,1.0);
        
        // ===== Create cone
        this.my_disc = new disc(this.gl, [0.9, 0, 0.5] ,[0, 1, 1], 7);
        this.my_disc.setShader(this.shaderprog);
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
   
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("DiscTex.png");
        
        this.my_discT = new discT(this.gl,[0.9, 0, 0] ,[1, 1, 0],30);
        this.my_discT.settexture(this.texture);
        this.my_discT.setShader(this.shaderprogT);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(4,1,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
////////////////// Transform/Animate Couple //////////////////
        var model = glm.mat4(glm.mat3(1.0));
         var translation = glm.vec3(0,0,-1);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 0.0, 1.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_disc.setModelTransformation(model);
        
        // ===== Draw cone
        this.my_disc.drawit(this.viewM,this.projectionM);
        
        var model = glm.mat4(glm.mat3(1.0));
         var translation = glm.vec3(0,0,1);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 0.0, 1.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_discT.setModelTransformation(model);
        
        // ===== Draw cone
        this.my_discT.drawit(this.viewM,this.projectionM);
    }
}

var app = new DiscApp('DiscCanvas');

app.run();
</script>
# Compound Shape
## Moon with Flag


<canvas id="MoonCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="MoonApp">

//////////////////////////////// FLAG //////////////////////////////// 
class flag extends CGRAobject 
{
    countermoon = 0;
    
    // ===== Constructor
    constructor(glcontext)
    {
        super(glcontext);
        
        
        var flagTex = new CGRAtexture(glcontext);
        var poleTex = new CGRAtexture(glcontext);
        flagTex.load("FlagTex.png");
        poleTex.load("woodTex.jpg");

        
        // ===== Instantiate components
        this.pole = new cylinderT(this.gl,[1, 1, 0.58] ,[1, 0.8, 0.2], 20);
        this.flag = new cubeT(this.gl);
        

        this.pole.settexture(poleTex);
        this.flag.settexture(flagTex);
    }
    
    // ===== Extend drawit
    drawit(viewM, projectM,parentMat4 = glm.mat4(1.0))
    {
        // ===== Falg global model matrix
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.pole.setShader(this.shaderprog);
        this.flag.setShader(this.shaderprog);
        
        // ===== Pole Model
        var pole = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.5, 3.1, 0.5);
        pole = glm.scale(pole, scale);
        // ===== Translate
        var translation = glm.vec3(0,0,0);
        pole = glm.translate(pole, translation);
        // ===== Final Transformation
        this.pole.setModelTransformation(pole);      
        
        // ===== Draw pole
        this.pole.drawit(viewM,projectM,globalMat);
        
        
        // ===== Flag Model
        var flag = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.01, 1, 2);
        flag = glm.scale(flag, scale);
        // ===== Translate
        var translation = glm.vec3(0,1,-0.5);
        flag = glm.translate(flag, translation);
        // ===== Final Transformation
        this.flag.setModelTransformation(flag);  
        
        // ===== Draw flag
        this.flag.drawit(viewM,projectM,globalMat);
    }
}

//////////////////////////////// Moon //////////////////////////////// 
class moon extends CGRAobject 
{
    counter = 0;
    
    // ===== Constructor
    constructor(glcontext)
    {
        super(glcontext);
        
        // ===== Instantiate components
        
         var moonTex = new CGRAtexture(glcontext);
        moonTex.load("MoonTex.jpg");
        
        this.moon = new sphereT(this.gl,[0.58, 0.58, 0.58] , 20);
        this.flag = new flag(this.gl);
        
        this.moon.settexture(moonTex);
        
    }
    
    // ===== Extend drawit
    drawit(viewM, projectM, parentMat4 = glm.mat4(1.0))
    {
        // ===== Calculate global model matrix
        var globalMat = parentMat4['*'](this.modelMat);
        
        this.counter ++;
        
        this.moon.setShader(this.shaderprog);
        this.flag.setShader(this.shaderprog);
        
        // ===== Moon Model
        var moon = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(2.5, 2.5, 2.5);
        moon = glm.scale(moon, scale);
        // ===== Translate
        var translation = glm.vec3(0,0,0);
        moon = glm.translate(moon, translation);
        // ===== Final Transformation
        this.moon.setModelTransformation(moon);      
        
        // ===== Draw moon
        this.moon.drawit(viewM,projectM,globalMat);

        
        // ===== Flag Model
        var model = glm.mat4(glm.mat3(0.25));
        // ===== Scale
        var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
        // ===== Rotation
        var rotation = glm.radians(-45);
        var axis = glm.vec3(1.0, 0.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0, 0.0, 1);
        model = glm.rotate(model, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,6,0);
        model = glm.translate(model, translation);
        // ===== Final Transformation
        this.flag.setModelTransformation(model);
        
        // ===== Draw flag
        this.flag.drawit(viewM,projectM,globalMat); 
    }
}


//////////////////////////////// SCENE //////////////////////////////// 
class MoonApp extends DEECapp
{
    counter=0;
    initialize()
    {
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
         var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        
        
        // ===== Create moon
        this.moon = new moon(this.gl);
        this.moon.setShader(this.shaderprogT);
                
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(5,0,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;

        this.shaderprog.startUsing();
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.moon.setModelTransformation(model);
        
        // ===== Draw
        this.moon.drawit(this.viewM,this.projectionM);     
    }
}

var app = new MoonApp('MoonCanvas');

app.run();
</script>


## Couple


<canvas id="CoupleCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="CoupleApp">

//////////////////////////////// COUPLE ////////////////////////////////
class couple extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext)
    {
        super(glcontext);
        
        // ===== Instantiate components
        this.woman = new woman(this.gl);
        this.man = new man(this.gl);
    }
    
        setShader(shader){
        this.man.setShader(shader);
        this.woman.setShader(shader);
    }
    
    // ===== Extend drawit
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {    
        var couplemodeMat = parentMat4['*'](this.modelMat);

        // ===== Draw components
        this.man.drawit(viewM,projectionM,couplemodeMat);
        this.woman.drawit(viewM,projectionM,couplemodeMat);
    }
}

//////////////////////////////// WOMAN ////////////////////////////////
class woman extends CGRAobject 
{
    // ===== Constructor
    constructor(glcontext)
    {
        super(glcontext);

        var headTex = new CGRAtexture(glcontext);
        var dressTex = new CGRAtexture(glcontext);
        headTex.load("HeadTex.png");
        dressTex.load("dressW.jpg");
        
        // ===== Instantiate components
        this.body = new coneT(this.gl);
        this.head = new sphereT(this.gl);
        this.head.settexture(headTex);
        this.body.settexture(dressTex);
    }
    
        setShader(shader){
        this.head.setShader(shader);
        this.body.setShader(shader);
    }
    
    // ===== Extend drawit
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        // ===== Connect to parent model
        var globalMat = parentMat4['*'](this.modelMat);
 
        // ===== Apply transformations to body
        var scale = glm.mat4(glm.mat3(1));
        var translate = glm.translate(glm.mat4(1.0),glm.vec3(0,0,0.5));
        var modelMat_body = translate["*"](scale);
        this.body.setModelTransformation(modelMat_body);
        this.body.drawit(viewM, projectionM, globalMat);
        
        // ===== Apply transformations to head
        var scale = glm.mat4(glm.mat3(0.5));
        var translate = glm.translate(glm.mat4(1.0),glm.vec3(0,0.7,0.5));
        var rotation = glm.toMat4(glm.angleAxis(glm.radians(180),glm.vec3(0.0,0.0,1.0)));
        var modelMat_head = translate["*"](rotation['*'](scale));
        this.head.setModelTransformation(modelMat_head);
        this.head.drawit(viewM, projectionM, globalMat);
    }
}

//////////////////////////////// MAN //////////////////////////////// 
class man extends CGRAobject 
{
    // ===== Constructor
    constructor(glcontext)
    {
        // Initialize the parent class
        super(glcontext); 
        
       
        var dressTex = new CGRAtexture(glcontext);
        var headTex = new CGRAtexture(glcontext);
        
        dressTex.load("dressM.jpg");
        headTex.load("HeadTex.png");
        
        // ===== Instantiate components
        this.head = new sphereT(glcontext);
        this.body = new coneT(glcontext);
        
        this.head.settexture(headTex);
        this.body.settexture(dressTex);
        
    }
    
    setShader(shader){
        this.head.setShader(shader);
        this.body.setShader(shader);
    }
    
    // ===== Extend drawit
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        // ===== Connect to parent model
        var globalMat = parentMat4['*'](this.modelMat);
        

        
        // ===== Apply transformations to body
        var scale = glm.mat4(glm.mat3(1));
        translate = glm.translate(glm.mat4(1.0),glm.vec3(0,0,-0.5));
        var rotation = glm.toMat4(glm.angleAxis(glm.radians(180),glm.vec3(0.0,0.0,1.0)));
        var modelMat_body = translate["*"](rotation['*'](scale));
        this.body.setModelTransformation(modelMat_body);
        this.body.drawit(viewM, projectionM, globalMat);
        
        // ===== Apply transformations to head
        var scale = glm.mat4(glm.mat3(0.5));
        var translate = glm.translate(glm.mat4(1.0), glm.vec3(0,0.7,-0.5));
        var rotation = glm.toMat4(glm.angleAxis(glm.radians(180),glm.vec3(0.0,0.0,1.0)));
        var modelMat_head = translate["*"](rotation['*'](scale));
        this.head.setModelTransformation(modelMat_head);
        this.head.drawit(viewM, projectionM, globalMat);
    }
}


//////////////////////////////// SCENE //////////////////////////////// 
class CoupleApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
   
        
        // ===== Create couple
        this.couple = new couple(this.gl);
        this.couple.setShader(this.shaderprogT);
        
        

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(5,0,0),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.couple.setModelTransformation(model);
      
        // ===== Draw couple
        this.couple.drawit(this.viewM,this.projectionM);   
    }
}

var app = new CoupleApp('CoupleCanvas');

app.run();
</script>


# Table


<canvas id="TableCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="TableApp">

//////////////////////////////// TABLE //////////////////////////////// 
class table extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [0.8,0.7,0.6], col2 =[0.6,0.3,0.1])
    {
        // ===== Initialize the parent class
        super(glcontext);
        
        var Tex = new CGRAtexture(glcontext);
        
        Tex.load("woodTex.jpg");
        
        // ===== Instantiate components
        this.top = new cubeT(this.gl, col1, col2);
        this.leg1 = new cylinderT(this.gl, col1, col2, 10);
        this.leg2 = new cylinderT(this.gl, col1, col2, 10);
        this.leg3 = new cylinderT(this.gl, col1, col2, 10);
        this.leg4 = new cylinderT(this.gl, col1, col2, 10)  ; 
        
              this.top.settexture(Tex);
              this.leg1.settexture(Tex);
              this.leg2.settexture(Tex);
              this.leg3.settexture(Tex);
                    this.leg4.settexture(Tex);
    }
    
    // ===== Extend drawit
    drawit(viewM, projectM, parentMat4 = glm.mat4(1.0))
    {
        // ===== Connect to parent model
        var globalMat = parentMat4['*'](this.modelMat);
        
        this.top.setShader(this.shaderprog);        
        this.leg1.setShader(this.shaderprog);
        this.leg2.setShader(this.shaderprog);
        this.leg3.setShader(this.shaderprog);
        this.leg4.setShader(this.shaderprog);
        
        // ================ Top ==================
        var top = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(2.5, 0.25, 2.5);
        top = glm.scale(top, scale);
        // ===== Translate
        var translation = glm.vec3(0,0,0);
        top = glm.translate(top, translation);
        // ===== Final Transformation
        this.top.setModelTransformation(top);      
        // ===== Draw
        this.top.drawit(viewM,projectM,globalMat);
        
        // ================ Leg1 ==================
        var leg1 = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.25, 1.5, 0.25);
        leg1 = glm.scale(leg1, scale);
        // ===== Translate
        var translation = glm.vec3(4,-0.5,4);
        leg1 = glm.translate(leg1, translation);
        // ===== Final Transformation
        this.leg1.setModelTransformation(leg1);      
        // ===== Draw
        this.leg1.drawit(viewM,projectM,globalMat);
        
        // ================ Leg2 ==================
        var leg2 = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.25, 1.5, 0.25);
        leg2 = glm.scale(leg2, scale);
        // ===== Translate
        var translation = glm.vec3(-4,-0.5,-4);
        leg2 = glm.translate(leg2, translation);
        // ===== Final Transformation
        this.leg2.setModelTransformation(leg2);      
        // ===== Draw
        this.leg2.drawit(viewM,projectM,globalMat);
        
        // ================ Leg3 ==================
        var leg3 = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.25, 1.5, 0.25);
        leg3 = glm.scale(leg3, scale);
        // ===== Translate
        var translation = glm.vec3(-4,-0.5,4);
        leg3 = glm.translate(leg3, translation);
        // ===== Final Transformation
        this.leg3.setModelTransformation(leg3);      
        // ===== Draw
        this.leg3.drawit(viewM,projectM,globalMat);
        
        // ================ Leg4 ==================
        var leg4 = glm.mat4(glm.mat3(1));
        /// ===== Scale
        var scale = glm.vec3(0.25, 1.5, 0.25);
        leg4 = glm.scale(leg4, scale);
        // ===== Translate
        var translation = glm.vec3(4,-0.5,-4);
        leg4 = glm.translate(leg4, translation);
        // ===== Final Transformation
        this.leg4.setModelTransformation(leg4);      
        // ===== Draw
        this.leg4.drawit(viewM,projectM,globalMat);
    }
}

//////////////////////////////// SCENE //////////////////////////////// 
class TableApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9,0.9,0.9,1.0);
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        
        // ===== Instantiate table
        this.table = new table(this.gl,[0.8,0.7,0.6],[0.6,0.3,0.1]);

        this.table.setShader(this.shaderprogT);

 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,100);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(3,-1,5),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Translate
        //var translation = glm.vec3(Math.cos(this.counter/10),0,Math.sin(this.counter/10));
        //var translation = glm.vec3(0,0,0);
        //var translation = glm.vec3(0,2*Math.sin(this.counter/20),0);
        //model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.table.setModelTransformation(model);  
        
        // ===== Draw
        this.table.drawit(this.viewM,this.projectionM);
    }
}

var app = new TableApp('TableCanvas');

app.run();
</script>
## Paintings


<canvas id="PaintingCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="PaintingApp">

class Painting1 extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
         
        var paintingTex = new CGRAtexture(glcontext);
        paintingTex.load("PaintTposeTex.jpg");
        
        var frameTex = new CGRAtexture(glcontext);
        frameTex.load("woodTex.jpg");
        
        this.frame = new cubeT(this.gl); 
        this.painting = new cubeT(this.gl)             
        
        this.painting.settexture(paintingTex);
        this.frame.settexture(frameTex);   
        
        var scale = glm.scale(glm.vec3(1.7,0.8,0.15));
        var translate = glm.translate(scale,glm.vec3(0,0,0))
        var Rotation = glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,0.0,1.0)));      
        var transformation = translate['*'](Rotation);
        this.painting.setModelTransformation(transformation);
        
        
        var scale = glm.scale(glm.vec3(2,1,0.1));
        var transformation = glm.translate(scale,glm.vec3(0,0,0));
        this.frame.setModelTransformation(transformation);
        
        
     }
        
    setShader(shader){ 
        this.painting.setShader(shader);
        this.frame.setShader(shader);
    }   
            
    drawit(viewMat, projectionMat, parentMat4 = glm.mat4(1.0)){
        var globalMat = parentMat4['*'](this.modelMat);
        this.painting.drawit(viewMat, projectionMat, globalMat);
        this.frame.drawit(viewMat, projectionMat, globalMat);            
    } 
        
}

class Painting2 extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
          
        var paintingTex = new CGRAtexture(glcontext);
        paintingTex.load("PaintRockTex.jpg");
        
        var frameTex = new CGRAtexture(glcontext);
        frameTex.load("woodTex.jpg");
        
        this.frame = new cubeT(this.gl); 
        this.painting = new cubeT(this.gl)             
        
        this.painting.settexture(paintingTex);
        this.frame.settexture(frameTex);   
        
        var scale = glm.scale(glm.vec3(1.7,0.8,0.15));
        var translate = glm.translate(scale,glm.vec3(0,0,0))
        var Rotation = glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,0.0,1.0)));      
        var transformation = translate['*'](Rotation);
        this.painting.setModelTransformation(transformation);
        
        
        var scale = glm.scale(glm.vec3(2,1,0.1));
        var transformation = glm.translate(scale,glm.vec3(0,0,0));
        this.frame.setModelTransformation(transformation);
        
        
     }
        
    setShader(shader){ 
        this.painting.setShader(shader);
        this.frame.setShader(shader);
    }   
            
    drawit(viewMat, projectionMat, parentMat4 = glm.mat4(1.0)){
        var globalMat = parentMat4['*'](this.modelMat);
        this.painting.drawit(viewMat, projectionMat, globalMat);
        this.frame.drawit(viewMat, projectionMat, globalMat);            
    } 
        
        
}



class PaintingApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9,0.9,0.9,1.0);
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        
        // ===== Instantiate table
        this.painting = new Painting1(this.gl);

        this.painting.setShader(this.shaderprogT);

        this.painting1 = new Painting2(this.gl);

        this.painting1.setShader(this.shaderprogT);

 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,100);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(0,0,5),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);

        
        this.shaderprog.startUsing();
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Translate
        var translation = glm.vec3(0,1,0);

        model = glm.translate(model, translation);
        // ===== Final Transformation
        this.painting.setModelTransformation(model);  
        
        // ===== Draw
        this.painting.drawit(this.viewM,this.projectionM);
        
         var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Translate
        var translation = glm.vec3(0,-1,0);

        model = glm.translate(model, translation);
        // ===== Final Transformation
        this.painting1.setModelTransformation(model);  
        
        // ===== Draw
        this.painting1.drawit(this.viewM,this.projectionM);
    }
}

var app = new PaintingApp('PaintingCanvas');

app.run();
</script>
## Museum = Floor +  Walls +  Door + Windows

W - Front | A - Left | S - Back | D - Right | . - Reset


<canvas id="MuseumCanvas" width="600" height="600" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="MuseumApp">

//////////////////////////////// WALL ////////////////////////////////
class wallRoom extends CGRAobject
{
    constructor(glcontext, color = [0.9, 0.9, 0.9])
    {
        super(glcontext);
        var Tex = new CGRAtexture(glcontext);
        
        Tex.load("WallTex.png");
        this.wallObj = new cubeT(this.gl);
        this.wallObj.settexture(Tex);
    }
    
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.wallObj.setShader(this.shaderprog);        
    
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.wallObj.setModelTransformation(model);
        this.wallObj.drawit(viewM, projectionM, globalMat);
    }
}


//////////////////////////////// FLOOR ////////////////////////////////

class floorRoom extends CGRAobject
{
        constructor(glcontext, color = [0.9, 0.9, 0.9])
    {
        super(glcontext);
        var Tex = new CGRAtexture(glcontext);
        
        Tex.load("FloorTex.png");
        this.wallObj = new cubeT(this.gl);
        this.wallObj.settexture(Tex);
    }
    
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.wallObj.setShader(this.shaderprog);        
    
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.wallObj.setModelTransformation(model);
        this.wallObj.drawit(viewM, projectionM, globalMat);
    }
}
//////////////////////////////// DOOR //////////////////////////////// 
class doorRoom extends CGRAobject
{
    constructor(glcontext, color = [0.9, 0.9, 0.9])
    {
        super(glcontext);
        
        // ===== Instantiate components
        var Tex = new CGRAtexture(glcontext);
        
        Tex.load("WallTex.png");
        
        this.left = new cubeT(this.gl);
        this.right = new cubeT(this.gl);
        this.top = new cubeT(this.gl);
        
        this.right.settexture(Tex);
        this.left.settexture(Tex);
        this.top.settexture(Tex);
    }
    
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        // ===== Connect to parent
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.left.setShader(this.shaderprog);
        this.right.setShader(this.shaderprog);
        this.top.setShader(this.shaderprog);
        
        ///////////// Left Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(-0.3,0,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.4, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.left.setModelTransformation(model);
        this.left.drawit(viewM, projectionM, globalMat);
        
        ///////////// Right Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0.3,0,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.4, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.right.setModelTransformation(model);
        this.right.drawit(viewM, projectionM, globalMat);
        
        ///////////// Top Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,0.2,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.2, 0.6, 0.05);
        model = glm.scale(model, scale);
        this.top.setModelTransformation(model);
        this.top.drawit(viewM, projectionM, globalMat);
    }
}

//////////////////////////////// WINDOW //////////////////////////////// 
class windowRoom extends CGRAobject
{
    constructor(glcontext, color = [0.9, 0.9, 0.9])
    {
        super(glcontext);
        
        // ===== Instantiate components
        
        var Tex = new CGRAtexture(glcontext);
        
        Tex.load("WallTex.png");
        
        this.left = new cubeT(this.gl);
        this.right = new cubeT(this.gl);
        this.top = new cubeT(this.gl);
        this.bottom = new cubeT(this.gl);
        
        this.right.settexture(Tex);
        this.left.settexture(Tex);
        this.top.settexture(Tex);
        this.bottom.settexture(Tex);

        
    }
    
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        // ===== Connect to parent
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.left.setShader(this.shaderprog);
        this.right.setShader(this.shaderprog);
        this.top.setShader(this.shaderprog);
        this.bottom.setShader(this.shaderprog);
        
        ///////////// Left Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(-0.35,0,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.3, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.left.setModelTransformation(model);
        this.left.drawit(viewM, projectionM, globalMat);
        
        ///////////// Right Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0.35,0,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.3, 1.0, 0.05);
        model = glm.scale(model, scale);
        this.right.setModelTransformation(model);
        this.right.drawit(viewM, projectionM, globalMat);
        
        ///////////// Top Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,0.25,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.4, 0.5, 0.05);
        model = glm.scale(model, scale);
        this.top.setModelTransformation(model);
        this.top.drawit(viewM, projectionM, globalMat);
        
        ///////////// Bottom Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,-0.4,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(0.4, 0.2, 0.05);
        model = glm.scale(model, scale);  
        this.bottom.setModelTransformation(model);
        this.bottom.drawit(viewM, projectionM, globalMat);
    }
}

//////////////////////////////// MUSEUM //////////////////////////////// 
class museumRoom extends CGRAobject
{
    constructor(glcontext, colorful = 0)
    {
        // ===== Initialize super
        super(glcontext);
        
        // ===== Instantiate components
        if(colorful)
        {
            this.back = new wallRoom(this.gl,[1,0,0]);
            this.front = new doorRoom(this.gl,[0,1,0]);
            this.left = new windowRoom(this.gl,[0,0,1]);
            this.right = new windowRoom(this.gl,[1,1,0]);
            this.floor = new floorRoom(this.gl,[0.9,0.9,0.9]);
        }
        else
        {
            this.back = new wallRoom(this.gl,[0.9, 0.9, 0.7]);
            this.front = new doorRoom(this.gl,[0.9, 0.9, 0.7]);
            this.left = new windowRoom(this.gl,[0.9, 0.9, 0.7]);
            this.right = new windowRoom(this.gl,[0.9, 0.9, 0.7]);
            this.floor = new floorRoom(this.gl,[0.9, 0.7, 0.4]);
        }        
    }
    
    // ===== Extend drawit
    drawit(viewM, projectionM, parentMat4 = glm.mat4(1.0))
    {   
        // ===== Connect to parent
        var globalMat = parentMat4['*'](this.modelMat);
 
        this.back.setShader(this.shaderprog);
        this.front.setShader(this.shaderprog);
        this.left.setShader(this.shaderprog);
        this.right.setShader(this.shaderprog);
        this.floor.setShader(this.shaderprog);
        
        ///////////// Back Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0.0, 0.0, -0.525);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
        this.back.setModelTransformation(model);
        this.back.drawit(viewM, projectionM, globalMat);
        
        ///////////// Front Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0.0, 0.0, 0.525);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(1.0);
        model = glm.scale(model, scale);
        this.front.setModelTransformation(model);
        this.front.drawit(viewM, projectionM, globalMat);
        
        ///////////// Left Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(-0.525,0,0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(90);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3([1.1, 1.0, 1.0]);
        model = glm.scale(model, scale);
        this.left.setModelTransformation(model);
        this.left.drawit(viewM, projectionM, globalMat);
        
        ///////////// Right Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0.525,0,0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(90);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3([1.1, 1.0, 1.0]);
        model = glm.scale(model, scale);  
        this.right.setModelTransformation(model);
        this.right.drawit(viewM, projectionM, globalMat);
        
        ///////////// Floor Part /////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,-0.525,0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(90);
        var axis = glm.vec3(1.0, 0.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3([1.1, 1.1, 1.0]);
        model = glm.scale(model, scale);  
        this.floor.setModelTransformation(model);
        this.floor.drawit(viewM, projectionM, globalMat);
    }
}

//////////////////////////////// SCENE //////////////////////////////// 
class MuseumApp extends DEECapp
{
    positionX = 0.0;
    positionZ = 5.0;
    camera_angle = glm.radians(270);
    counter = 0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
                var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        

        
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.1,0.1,0.1,1.0);
        
        // ===== Create museum object
        var colorful = 0;
        this.building = new museumRoom(this.gl, colorful);
        this.building.setShader(this.shaderprogT);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(100),1.0,0.001,10000);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(0,0,3),glm.vec3(0,0,0),glm.vec3(0,1,0));           
        
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
    // ===== Process key presses
    keyprocess(evt)
    {
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 100: // D - Right
                 this.camera_angle = this.camera_angle + 0.1;
                 break;
             case 97:  // A - Left
                 this.camera_angle = this.camera_angle - 0.1;
                 break;
             case 115: // S - Back
                 this.positionZ-= 20*Math.sin(this.camera_angle);
                 this.positionX -= 20*Math.cos(this.camera_angle);
                 break;
             case 119: // W - Front
                 this.positionZ+= 20*Math.sin(this.camera_angle);;
                 this.positionX += 20*Math.cos(this.camera_angle);
                 break;
             case 46:  // . - Reset camera position
                 this.positionZ = 5;
                 this.positionX = 0;
                 this.camera_angle = glm.radians(270);
    }
}
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        // ===== Allow movement of "player"
        this.viewM = glm.lookAt(glm.vec3(this.positionX,0.5,this.positionZ),
                                glm.vec3(this.positionX + Math.cos(this.camera_angle),
                                         0.5,
                                         this.positionZ + Math.sin(this.camera_angle)),
                                glm.vec3(0,1,0));        
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,100,0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        //model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3(500.0);
        model = glm.scale(model, scale);
        // ===== Final Transformation
        this.building.setModelTransformation(model);
        
        // ===== Draw museum
        this.building.drawit(this.viewM,this.projectionM); 
    }
}

var app = new MuseumApp('MuseumCanvas');

app.run();
</script>


## Solar System

1 - Stop | 2 - Slow | 3 - Normal | 4 - Fast

R - Spheres | T - Cubes | Y - Cylinders | U - Cones


<canvas id="myCCanvas_solarSystem" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myCapp_solarSystem">

class solarSystem extends CGRAobject
{
    form = 1;
    constructor(glcontext, form = 1, sunFlg = 1)
    {    
        super(glcontext)
    

        this.form = form;
        
        
        var SunTex = new CGRAtexture(glcontext);
        var MerTex = new CGRAtexture(glcontext);
        var VenTex = new CGRAtexture(glcontext);
        var EarTex = new CGRAtexture(glcontext);
        var MarTex = new CGRAtexture(glcontext);
        var JupTex = new CGRAtexture(glcontext);
        var SatTex = new CGRAtexture(glcontext);
        var SatDTex = new CGRAtexture(glcontext);
        var UraTex = new CGRAtexture(glcontext);
        var NepTex = new CGRAtexture(glcontext);
        


        
        SunTex.load("SunTex.jpg");
        MerTex.load("MerTex.jpg");
        VenTex.load("VenTex.jpg");
        EarTex.load("EarTex.jpg");
        MarTex.load("MarTex.jpg");
        JupTex.load("JupTex.jpg");
        SatTex.load("SatTex.jpg");
        SatDTex.load("SatDTex.png");
        UraTex.load("UraTex.jpg");
        NepTex.load("NepTex.jpg");
        
        

        
        switch(form)
        {
            case 1: 
                this.Sun = new sphereTC(this.gl,[1, 0.7, 0] , 20);
                this.Mercury = new sphereT(this.gl,[0.58, 0.58, 0.58] , 20);
                this.Venus = new sphereT(this.gl,[0.58, 0.29, 0.1] , 20);
                this.Earth = new sphereT(this.gl,[0, 0.8, 0.3], 20);
                this.Mars = new sphereT(this.gl,[0.9, 0, 0], 20);
                this.Jupiter = new sphereT(this.gl,[0.9, 0.4, 0], 20);
                this.Saturn = new sphereT(this.gl,[0.8, 0.4, 0], 20);
                this.Uranus = new sphereT(this.gl,[0.4, 0.6, 0.9], 20);
                this.Neptune = new sphereT(this.gl,[0, 0, 1], 20);
                break;
            case 2:
                this.Sun = new cubeTC(this.gl);
                this.Mercury = new cubeT(this.gl);
                this.Venus = new cubeT(this.gl);
                this.Earth = new cubeT(this.gl);
                this.Mars = new cubeT(this.gl);
                this.Jupiter = new cubeT(this.gl);
                this.Saturn = new cubeT(this.gl);
                this.Uranus = new cubeT(this.gl);
                this.Neptune = new cubeT(this.gl);
                break;
            case 3:
                this.Sun = new cylinderTC(this.gl,[1, 0.7, 0] ,[1, 1, 0], 10);
                this.Mercury = new cylinderT(this.gl,[0.58, 0.58, 0.58] ,[0.8, 0.8, 0.8], 10);
                this.Venus = new cylinderT(this.gl,[0.58, 0.29, 0.1] ,[1, 0.8, 0.67], 10);
                this.Earth = new cylinderT(this.gl,[0, 0.8, 0.3], [0, 0, 1], 10);
                this.Mars = new cylinderT(this.gl,[0.9, 0, 0] ,[1, 0.5, 0], 10);
                this.Jupiter = new cylinderT(this.gl,[0.9, 0.4, 0] ,[0.8, 0.8, 0.8], 10);
                this.Saturn = new cylinderT(this.gl,[0.8, 0.4, 0] ,[0.8, 0.8, 0.8], 10);
                this.Uranus = new cylinderT(this.gl,[0.4, 0.6, 0.9] ,[0, 0.7, 1], 10);
                this.Neptune = new cylinderT(this.gl,[0, 0, 1] ,[0, 0.5, 1], 10);
                break;
            case 4:
                this.Sun = new coneTC(this.gl,[1, 0.7, 0] ,[1, 1, 0], 10);
                this.Mercury = new coneT(this.gl,[0.58, 0.58, 0.58] ,[0.8, 0.8, 0.8], 10);
                this.Venus = new coneT(this.gl,[0.58, 0.29, 0.1] ,[1, 0.8, 0.67], 10);
                this.Earth = new coneT(this.gl,[0, 0.8, 0.3], [0, 0, 1], 10);
                this.Mars = new coneT(this.gl,[0.9, 0, 0] ,[1, 0.5, 0], 10);
                this.Jupiter = new coneT(this.gl,[0.9, 0.4, 0] ,[0.8, 0.8, 0.8], 10);
                this.Saturn = new coneT(this.gl,[0.8, 0.4, 0] ,[0.8, 0.8, 0.8], 10);
                this.Uranus = new coneT(this.gl,[0.4, 0.6, 0.9] ,[0, 0.7, 1], 10);
                this.Neptune = new coneT(this.gl,[0, 0, 1] ,[0, 0.5, 1], 10);
                break;
            }

            var Color = [0.3, 0.3, 0.3];
            switch(sunFlg){
                case 1:
                    Color = [0.3, 0.3, 0.3];
                break;
                case 2:
                    Color = [0.9, 0.9, 0.9];
                break;
                case 3:
                    Color = [1, 0.7, 0.0];
                break;
                case 4:
                    Color = [1, 0.0, 0.0];
                break;
            }
        
        this.SaturnDisc = new discT(this.gl, [0.9, 0.7, 0] ,[0.7, 0.7, 0.6], 20);
        
        this.Sun.settexture(SunTex);
        this.Mercury.settexture(MerTex);
        this.Venus.settexture(VenTex);
        this.Earth.settexture(EarTex);
        this.Mars.settexture(MarTex);
        this.Jupiter.settexture(JupTex);
        this.Saturn.settexture(SatTex);
        this.Uranus.settexture(UraTex);
        this.Neptune.settexture(NepTex);
        this.SaturnDisc.settexture(SatDTex);

       this.Sun.setcolor(Color)
            
        }
        
    setShader(shaderT,shaderTC){
        this.Sun.setShader(shaderTC);
        this.Mercury.setShader(shaderT);
        this.Venus.setShader(shaderT);
        this.Earth.setShader(shaderT);
        this.Mars.setShader(shaderT);
        this.Jupiter.setShader(shaderT);
        this.Saturn.setShader(shaderT);
        this.Uranus.setShader(shaderT);
        this.Neptune.setShader(shaderT);
        this.SaturnDisc.setShader(shaderT);
    }
    
    drawit(viewM, projectM, counter, parentMat4 = glm.mat4(1.0))
    {
        var globalMat = parentMat4['*'](this.modelMat);
        
       
        
        
        ////////////////////////SUN//////////////////////////////
         var Sun = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(2.5,2.5, 2.5);
        Sun = glm.scale(Sun, scale);
        // ===== Rotation
        var rotation = glm.radians(-counter);
        var axis = glm.vec3(0.0,1.0,0.0);
        Sun = glm.rotate(Sun, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,0);
        Sun = glm.translate(Sun, translation);
        // ===== Final Transformation
        this.Sun.setModelTransformation(Sun);      
        // ===== Draw
        this.Sun.drawit(viewM,projectM, globalMat);
        
        
        //////////////////////MERCURY///////////////////////////////////
        var Mercury = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.024, 0.024, 0.024);
        Mercury = glm.scale(Mercury, scale);
        // ===== Rotation
        var outer_rotation = counter*365/85;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Mercury = glm.rotate(Mercury, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-70);
        Mercury = glm.translate(Mercury, translation);
        // ===== Rotation
        Mercury = glm.rotate(Mercury, rotation, axis);
        // ===== Final Transformation
        this.Mercury.setModelTransformation(Mercury);      
        // ===== Draw
        this.Mercury.drawit(viewM,projectM, globalMat);
        
        
        /////////////////////VENUS/////////////////////////////////////
        var Venus = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.121/2, 0.121/2, 0.121/2);
        Venus = glm.scale(Venus, scale);
        // ===== Rotation
        var outer_rotation = counter*365/224;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Venus = glm.rotate(Venus, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-40);
        Venus = glm.translate(Venus, translation);
        // ===== Rotation
        var inner_rotation = counter/365;
        var rotation = glm.radians(inner_rotation);
        Venus = glm.rotate(Venus, rotation, axis);
        // ===== Final Transformation
        this.Venus.setModelTransformation(Venus);      
        // ===== Draw
        this.Venus.drawit(viewM,projectM, globalMat);
        
        
        ///////////////////////EARTH/////////////////////////////////////
        var Earth = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.127/2, 0.127/2, 0.127/2);
        Earth = glm.scale(Earth, scale);
        // ===== Rotation
        var outer_rotation = counter*365/365;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Earth = glm.rotate(Earth, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-50);
        Earth = glm.translate(Earth, translation);
        // ===== Rotation
        var inner_rotation = counter/365;
        var rotation = glm.radians(inner_rotation);
        Earth = glm.rotate(Earth, rotation, axis);
        // ===== Final Transformation
        this.Earth.setModelTransformation(Earth);      
        // ===== Draw
        this.Earth.drawit(viewM,projectM, globalMat);
        if(this.form == 1)
            this.Earth.update_pos(globalMat);
        
        
        ///////////////////////MARS/////////////////////////////////
        var Mars = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.067/2, 0.067/2, 0.067/2);
        Mars = glm.scale(Mars, scale);
        // ===== Rotation
        var outer_rotation = counter*365/(365*1.88);
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Mars = glm.rotate(Mars, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-120);
        Mars = glm.translate(Mars, translation);
        // ===== Rotation
        Mars = glm.rotate(Mars, rotation, axis);
        // ===== Final Transformation
        this.Mars.setModelTransformation(Mars);      
        // ===== Draw
        this.Mars.drawit(viewM,projectM, globalMat);
        
        
        //////////////////////JUPITER///////////////////////////////
        var Jupiter = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(1.42/2, 1.42/2, 1.42/2);
        Jupiter = glm.scale(Jupiter, scale);
        // ===== Rotation
        var outer_rotation = counter/11.86;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Jupiter = glm.rotate(Jupiter, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-8);
        Jupiter = glm.translate(Jupiter, translation);
        // ===== Rotation
        Jupiter = glm.rotate(Jupiter, rotation, axis);
        // ===== Final Transformation
        this.Jupiter.setModelTransformation(Jupiter);      
        // ===== Draw
        this.Jupiter.drawit(viewM,projectM, globalMat);
        
        
        //////////////////////SATURN///////////////////////////////
     
        var Saturn = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(1.2/2, 1.2/2, 1.2/2);
        Saturn = glm.scale(Saturn, scale);
        // ===== Rotation
        var outer_rotation = counter/29.46;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Saturn = glm.rotate(Saturn, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-12);
        Saturn = glm.translate(Saturn, translation);
        // ===== Rotation
        Saturn = glm.rotate(Saturn, rotation, axis);
        // ===== Final Transformation
        this.Saturn.setModelTransformation(Saturn);
        var scale = glm.vec3(1.5, 1.5, 1.5);
        Saturn = glm.scale(Saturn, scale);
        this.SaturnDisc.setModelTransformation(Saturn);
        
        // ===== Draw
        this.Saturn.drawit(viewM,projectM, globalMat);
        this.SaturnDisc.drawit(viewM,projectM, globalMat);
        
        
        
        //////////////////////URANUS/////////////////////////////
        var Uranus = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.51/2, 0.51/2, 0.51/2);
        Uranus = glm.scale(Uranus, scale);
        // ===== Rotation
        var outer_rotation = counter/84.01;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Uranus = glm.rotate(Uranus, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-36);
        Uranus = glm.translate(Uranus, translation);
        // ===== Rotation
        Uranus = glm.rotate(Uranus, rotation, axis);
        // ===== Final Transformation
        this.Uranus.setModelTransformation(Uranus);      
        // ===== Draw
        this.Uranus.drawit(viewM,projectM, globalMat);
        
        
        /////////////////////NEPTUNE////////////////////////////
        var Neptune = glm.mat4(glm.mat3(1));
        // ===== Scale
        var scale = glm.vec3(0.495/2, 0.495/2, 0.495/2);
        Neptune = glm.scale(Neptune, scale);
        // ===== Rotation
        var outer_rotation = counter/164.79;
        var rotation = glm.radians(outer_rotation);
        var axis = glm.vec3(0.0,1.0,0.0);
        Neptune = glm.rotate(Neptune, rotation, axis);
        // ===== Translate
        var translation = glm.vec3(0,0,-44);
        Neptune = glm.translate(Neptune, translation);
        // ===== Rotation
        Neptune = glm.rotate(Neptune, rotation, axis);
        // ===== Final Transformation
        this.Neptune.setModelTransformation(Neptune);      
        // ===== Draw
        this.Neptune.drawit(viewM,projectM, globalMat);  
    }
}


class myCapp_solarSystem extends DEECapp{
    counter=0;
    speedup = 1;
    SunFlag = 1;
    initialize(){
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
         var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        

        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);

        var fragsrcTC = document.getElementById("my-fragment-shaderTC").text;
        this.shaderprogTC = new DEECshader(this.gl);
        this.shaderprogTC.srcShaders(vertsrcT,fragsrcTC);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.1,0.1,0.1,1.0);
        
        this.solarSystem = new solarSystem(this.gl, 1);
        this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionMa= glm.perspective(glm.radians(60),1.0,0.1,1000);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(0, 0, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, -1): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewMa = glm.lookAt(glm.vec3(0,0,5),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
    keyprocess(evt)
    {
        //console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 49: // 1 Stop
                 this.speedup = 0;     
                 break;
             case 50: // 2 Slow
                 this.speedup = 0.2;
                 break;
             case 51: // 3 Middle
                 this.speedup = 1;
                 break;
             case 52: // 4 Fast
                 this.speedup = 10;
                 break;
             case 114: // r
             this.SunFlag = 1;
                 this.solarSystem = new solarSystem(this.gl, 1, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 116: // t
                this.SunFlag = 2;
                 this.solarSystem = new solarSystem(this.gl, 2, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 121: // y
                this.SunFlag = 3;
                 this.solarSystem = new solarSystem(this.gl, 3, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 117: // u
                this.SunFlag = 4;
                 this.solarSystem = new solarSystem(this.gl, 4, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
        }
    }
    
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter = this.counter + this.speedup;
        
        this.shaderprog.startUsing();
        
        // Draw
        this.solarSystem.drawit(this.viewMa,this.projectionMa, this.counter);        
    }
}

var app2 = new myCapp_solarSystem('myCCanvas_solarSystem');

app2.run();
</script>
# Solar System Scene

W - Front | A - Left | S - Back | D - Right | . - Reset

1 - Stop | 2 - Slow | 3 - Normal | 4 - Fast

R - Spheres | T - Cubes | Y - Cylinders | U - Cones


<canvas id="SolarSystemScene" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="SolarSystemScene">


class SolarSystemScene extends DEECapp{
    counter=0;
    speedup = 1;
    positionX = -5.0;
    positionZ = 0.0;
    camera_angle = 0;
    SunFlag = 1;
    initialize()
    {
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
         var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);


        var fragsrcTC = document.getElementById("my-fragment-shaderTC").text;
        this.shaderprogTC = new DEECshader(this.gl);
        this.shaderprogTC.srcShaders(vertsrcT,fragsrcTC);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.1,0.1,0.1,1.0);
        
        // Lets create solar system
        this.solarSystem = new solarSystem(this.gl, 1);
        this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
        
        
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(90),1.0,0.1,1000);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(0, 0, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, -1): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
                  
    
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
     keyprocess(evt)
    {
        //console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 49: // 1 Stop
                 this.speedup = 0;
                 break;
             case 50: // 2 Slow
                 this.speedup = 0.2;
                 break;
             case 51: // 3 Middle
                 this.speedup = 1;
                 break;
             case 52: // 4 Fast
                 this.speedup = 10;
                 break;
             case 114: // r
                 this.solarSystem = new solarSystem(this.gl, 1, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 116: // t
                 this.solarSystem = new solarSystem(this.gl, 2, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 121: // y
                 this.solarSystem = new solarSystem(this.gl, 3, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 117: // u
                 this.solarSystem = new solarSystem(this.gl, 4, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 100: //d
                 this.camera_angle = this.camera_angle + 0.1;
                 //console.log("camera angle: " + this.camera_angle);
                 break;
             case 97: //a
                 this.camera_angle = this.camera_angle - 0.1;
                 //console.log("camera angle: " + this.camera_angle);
                 break;
             case 115: // s
                 this.positionZ-= 0.2*Math.sin(this.camera_angle);
                 this.positionX -= 0.2*Math.cos(this.camera_angle);
                 //console.log("Position X: " + this.positionX);
                 //console.log("Position Z: " + this.positionZ);
                 break;
             case 119: //w
                 this.positionZ+= 0.2*Math.sin(this.camera_angle);;
                 this.positionX += 0.2*Math.cos(this.camera_angle);
                 //console.log("Position X: " + this.positionX);
                 //console.log("Position Z: " + this.positionZ);
                 break;
             case 46: //. reset camera position
                 this.positionZ = 0;
                 this.positionX = -5;
                 this.camera_angle = 0;
             
             
        }
    }
    
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter = this.counter + this.speedup;
        
        this.shaderprog.startUsing();
        
        this.viewM = glm.lookAt(glm.vec3(this.positionX,0.5,this.positionZ),glm.vec3(this.positionX + Math.cos(this.camera_angle),0.5,this.positionZ + Math.sin(this.camera_angle)),glm.vec3(0,1,0)); 
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(1.0, 1.0, 1.0);
        model = glm.scale(model, scale);
        // ===== Translate
        //var translation = glm.vec3(Math.cos(this.counter/10),0,Math.sin(this.counter/10));
        //var translation = glm.vec3(0,0,0);
        //var translation = glm.vec3(0,1*Math.sin(this.counter/20),0);
        //model = glm.translate(model, translation);
        // ===== Rotation
        //var rotation = glm.radians(this.counter*3);
        //var axis = glm.vec3(0.0, 1.0, 0.0);
        //model = glm.rotate(model, rotation, axis);
        
        // ===== Final Transformation
        this.solarSystem.setModelTransformation(model);  
        
        // ===== Draw
        this.solarSystem.drawit(this.viewM,this.projectionM, this.counter);
        
    }
}

var app = new SolarSystemScene('SolarSystemScene');

app.run();
</script>
# Final Scene

W - Front | A - Left | S - Back | D - Right | . - Reset

1 - Stop | 2 - Slow | 3 - Normal | 4 - Fast (Only applies to Solar System)

R - Spheres | T - Cubes | Y - Cylinders | U - Cones

M - Day | N - Night

Contains:
- 1 solar system
- 1 table 
- 1 moon with flag
- 1 guard (man)
- 1 couple
- 1 museum room
- 2 paintings


<canvas id="myScene" width="900" height="900" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myScene">


class myScene extends DEECapp{
    counter = 0;
    orbits = 0;
    speedup = 1;
    positionX = -15;
    positionZ = 15;
    camera_angle = glm.radians(-45);
    positionY = 0.033;
    view_flag = 1;
    SunFlag = 1;
    dayflg = 1;
    dayTex = new CGRAtexture(this.gl); 

    initialize()
    {
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);


        var fragsrcTC = document.getElementById("my-fragment-shaderTC").text;
        this.shaderprogTC = new DEECshader(this.gl);
        this.shaderprogTC.srcShaders(vertsrcT,fragsrcTC);
   
        
        // Initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.1,0.5,0.9,1.0);
        
        // Create Objects
        this.solarSystem = new solarSystem(this.gl);
        this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
        
        this.museum = new museumRoom(this.gl);
        this.museum.setShader(this.shaderprogT);
        
        this.expositor = new table(this.gl,[0.3,0.1,0.05]);
        this.expositor.setShader(this.shaderprogT);
        
        this.moon = new moon(this.gl);
        this.moon.setShader(this.shaderprogT);
        
        this.guard = new man(this.gl);
        this.guard.setShader(this.shaderprogT);
        
        this.couple = new couple(this.gl);
        this.couple.setShader(this.shaderprogT);
        
        this.painting = new Painting1(this.gl);
        this.painting.setShader(this.shaderprogT);

        this.painting1 = new Painting2(this.gl);
        this.painting1.setShader(this.shaderprogT);
        
        var grassTex = new CGRAtexture(this.gl);  
        grassTex.load("GrassTex.jpg");
        
        this.grass = new cubeT(this.gl);
        this.grass.setShader(this.shaderprogT);
        
        this.grass.settexture(grassTex);
        
      
        
        this.day = new sphereT(this.gl);
        this.day.setShader(this.shaderprogT);
        
       
        
        
        
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(100),1.0,0.001,1000);         
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(0, 0, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, -1): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////     
    
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
     keyprocess(evt)
    {
        //console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 49: // 1 Stop
                 this.speedup = 0;
                 break;
             case 50: // 2 Slow
                 this.speedup = 0.2;
                 break;
             case 51: // 3 Middle
                 this.speedup = 1;
                 break;
             case 52: // 4 Fast
                 this.speedup = 10;
                 break;
            case 114: // r
                 this.solarSystem = new solarSystem(this.gl, 1, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 116: // t
                 this.solarSystem = new solarSystem(this.gl, 2, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 121: // y
                 this.solarSystem = new solarSystem(this.gl, 3, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 117: // u
                 this.solarSystem = new solarSystem(this.gl, 4, this.SunFlag);
                 this.solarSystem.setShader(this.shaderprogT, this.shaderprogTC);
                 break;
             case 100: //d
                 this.camera_angle = this.camera_angle + 0.1;
                 break;
             case 97: //a
                 this.camera_angle = this.camera_angle - 0.1;
                 break;
             case 115: // s
                 this.positionZ-= 2*Math.sin(this.camera_angle);
                 this.positionX -= 2*Math.cos(this.camera_angle);
                 break;
             case 119: //w
                 this.positionZ+= 2*Math.sin(this.camera_angle);;
                 this.positionX += 2*Math.cos(this.camera_angle);
                 break;
             
             case 118: //v
                 if(this.view_flag == 1)
                     this.view_flag = 0;
                 else if(this.view_flag == 0)
                     this.view_flag = 1;
                 
                 break;

             case 46: //. reset camera position
                 if (this.view_flag == 1)
                 {
                     this.positionZ = 15;
                     this.positionX = -15;
                     this.camera_angle = glm.radians(-45);
                 }
                 break;
             case 109: // m
                 this.dayflg = 1;
                 break;
             case 110: // n
                 this.dayflg = 0;
        }
    }
    
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        if(this.dayflg == 1){
            this.dayTex.load("DayTex.jpg");
            this.day.settexture(this.dayTex);
        }
        else{
            this.dayTex.load("NightTex.jpg");
            this.day.settexture(this.dayTex);
        }
        // ===== Update Speed
        this.orbits = this.orbits + this.speedup;
        this.counter++;
        
        this.shaderprog.startUsing();
        
        ////////////////// Transform/Animate Solar System //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        //var translation = glm.vec3(0, 0, 0);
        //model = glm.translate(model, translation);
        // ===== Scale
        //var scale = glm.vec3(1);
        //model = glm.scale(model, scale);
        // ===== Final Transformation
        this.solarSystem.setModelTransformation(model);  
        
        ////////////////// Transform/Animate Museum //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,5,0);
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3([40.0, 20.0, 40.0]);
        model = glm.scale(model, scale);
        // ===== Final Transformation
        this.museum.setModelTransformation(model);
        
        ////////////////// Transform/Animate Expositor //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(0,-2,0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        //model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3([1, 2, 1]);
        model = glm.scale(model, scale);
        // ===== Final Transformation
        this.expositor.setModelTransformation(model);
        
        ////////////////// Transform/Animate Moon //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(-15, 2*Math.sin(this.counter/10), -15);
        model = glm.translate(model, translation);
        //var rotation = glm.radians(this.counter*5);
        //var axis = glm.vec3(0.0, 1.0, 0.0);
        //model = glm.rotate(model, rotation, axis);
        // ===== Scale
        //var scale = glm.vec3(1);
        //model = glm.scale(model, scale);
        // ===== Final Transformation
        this.moon.setModelTransformation(model);
        
        ////////////////// Transform/Animate Guard //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translate
        var translation = glm.vec3(15 - 2*(Math.sin(this.counter/15)+1), -2, 15 - 2*(Math.sin(this.counter/10)+1));
        model = glm.translate(model, translation);
        // ===== Scale
        var scale = glm.vec3(4);
        model = glm.scale(model, scale);
        // ===== Final Transformation
        this.guard.setModelTransformation(model);
        
        ////////////////// Transform/Animate Couple //////////////////
        var model = glm.mat4(glm.mat3(1.0));
        var translation = glm.vec3(3*Math.sin(this.counter/20), 0, 3*Math.sin(this.counter/20));
        model = glm.translate(model, translation);
        // ===== Translate
        var translation = glm.vec3(-9, -2, 9);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Scale
        var scale = glm.vec3(3.5);
        model = glm.scale(model, scale);
        // ===== Final Transformation
        this.couple.setModelTransformation(model);
        
        
         ////////////////// Transform Painting 1 //////////////////
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(4.0, 4.0, 4.0);
        model = glm.scale(model, scale);
        // ===== Translate
        var translation = glm.vec3(2,0,-5);

        model = glm.translate(model, translation);
        // ===== Final Transformation
        this.painting.setModelTransformation(model);  
        
        
        
        ////////////////// Transform Painting 2 //////////////////
         var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(4.0, 4.0, 4.0);
        model = glm.scale(model, scale);
        // ===== Translate
        var translation = glm.vec3(-2,0,-5);

        model = glm.translate(model, translation);
        // ===== Final Transformation
        this.painting1.setModelTransformation(model);  

        
         ////////////////// Trasform Grass//////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(500.0, 1, 500.0);
        model = glm.scale(model, scale);
        // ===== Translate
        var translation = glm.vec3(0,-10,0);

        model = glm.translate(model, translation);
        // ===== Final Transformation
        this.grass.setModelTransformation(model);  
        ////////////////// Trasform SkyDay//////////////////
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Scale
        var scale = glm.vec3(500.0, 500, 500.0);
        model = glm.scale(model, scale);
        // ===== Translate
        var translation = glm.vec3(0,0,0);
        
        var rotation = glm.radians(this.counter/10);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);

        model = glm.translate(model, translation);
        // ===== Final Transformation
        this.day.setModelTransformation(model);  
        
        
        if(this.view_flag == 1)
        {
            ////////////////// Visiter View //////////////////
            this.viewM = glm.lookAt(glm.vec3(this.positionX, 0.5, this.positionZ),
                                    glm.vec3(this.positionX + Math.cos(this.camera_angle), 
                                             0.5, 
                                             this.positionZ + Math.sin(this.camera_angle)),
                                    glm.vec3(0,1,0)); 
        }
        else
        {   
            ////////////////// Planet View //////////////////
            var Earth = glm.mat4(glm.mat3(1));
            // ===== Scale
            var scale = glm.vec3(0.127/2, 0.127/2, 0.127/2);
            Earth = glm.scale(Earth, scale);
            // ===== Rotation
            var outer_rotation = this.orbits;
            var rotation = glm.radians(outer_rotation);
            var axis = glm.vec3(0.0,1.0,0.0);
            Earth = glm.rotate(Earth, rotation, axis);
            // ===== Translate
            var translation = glm.vec3(0,0,-50);
            Earth = glm.translate(Earth, translation);
            // ===== Rotation
            var inner_rotation = this.orbits;
            var rotation = glm.radians(-inner_rotation);
            Earth = glm.rotate(Earth, rotation, axis);
            var axis = glm.vec3(0.0,1.0,0.0);
            Earth = glm.rotate(Earth, rotation, axis);   
            
            //var center = this.solarSystem.Earth.center;
            //console.log("Center: " + center);
            //var north = this.solarSystem.Earth.north_pole;
            //console.log("North: " + north);
            
            //this.viewM = glm.lookAt(glm.vec3(center[0], center[1], center[2]),
            //                        glm.vec3(0,this.positionY,0),
            //                        glm.vec3(0,1,0));
            
            this.viewM = glm.lookAt(glm.vec3(Earth[3][0], this.positionY,Earth[3][2]),
                                    glm.vec3(0,this.positionY,0),
                                    glm.vec3(0,1,0));
        }
        
        
        // ===== Draw
        this.solarSystem.drawit(this.viewM,this.projectionM, this.orbits);
        this.museum.drawit(this.viewM,this.projectionM);
        this.expositor.drawit(this.viewM,this.projectionM);
        this.moon.drawit(this.viewM,this.projectionM);
        this.guard.drawit(this.viewM,this.projectionM);
        this.couple.drawit(this.viewM,this.projectionM);
        this.painting.drawit(this.viewM,this.projectionM);
        this.painting1.drawit(this.viewM,this.projectionM);
        this.grass.drawit(this.viewM,this.projectionM);
        this.day.drawit(this.viewM,this.projectionM);
        
        
    }
}

var app2 = new myScene('myScene');

app2.run();
</script>
